# Libraries

In [1]:
import sys
import os
%matplotlib inline

In [2]:
os.getcwd()

'c:\\Users\\gabi1\\git\\ACTC_MD_ML\\gaby_notebooks'

In [3]:
os.chdir('..\\')
os.getcwd()

'c:\\Users\\gabi1\\git\\ACTC_MD_ML'

In [4]:
from rdkit.Chem.Draw import rdMolDraw2D
from src.trainers import Trainers
from src.polymerSMILES import polymers
from src.dataProcessing import * #DataPreprocessing #computeTanimotoSimilarity #* #data_load, computeTanimotoSimilarity, get_dissimilarity_map, featurizer, normalizedata
from src.solver import *

# Data preprocessing part

### Load polymer smiles

In [5]:
# load smiles
polymers_dict = polymers.get_data()

### Implement datapreprocessing class

In [6]:
# data preprocessing toolbox
preprocess = DataPreprocessing()

In [8]:
#loading the data
data = preprocess.data_load(filename = "1.0-hkg-MD-ActCoeff-processed.csv")
data.head()

,#,Name of the polymer,CounterIon,Co-Ion,salt,Water-per-ion,concentration_of_salt_M,gr_minima_Ion_H2O,gr_peak_position_Ion_H2O,gr_peak_height_Ion_H2O,Nr_Ion_H2O,gr_minima_CG_H2O,gr_peak_position_CG_H2O,gr_peak_height_CG_H2O,Nr_CG_H2O_,gr_minima_CG_Ion,gr_peak_position_CG_Ion,gr_peak_height_CG_Ion,Nr_CG_Ion,Exp_act_coeff
0,1,PSbNMP_RCE,I-,K+,KI,6,0.00,4.45,3.65,6.5,4.7,5.4,3.55,2.16,5.3,6.25,4.15,5.6,1.9,-
1,2,PSbNMP_RCE,I-,K+,KI,6,0.02,4.45,3.65,6.5,4.7,5.5,3.55,2.30,6.1,6.15,4.15,5.8,1.9,0.300
2,3,PSbNMP_RCE,I-,K+,KI,6,0.20,4.45,3.65,6.3,4.6,5.5,3.55,1.90,5.0,6.15,4.15,5.8,2.0,0.319
3,4,PSbNMP_RCE,I-,K+,KI,6,0.50,4.45,3.65,6.2,4.6,5.5,3.55,1.80,4.5,6.15,4.15,5.6,2.2,0.476
4,5,PSbNMP_RCE,I-,K+,KI,6,1.00,4.55,3.65,6.0,4.7,5.0,3.55,1.50,3.0,6.15,4.15,5.1,2.5,0.663


### Featurizer the data 

In [10]:
(X, y, MFprints) = preprocess.featurizer(data, polymers_dict,  'solvation') # descr = 'solvation' or 'activity'

In [12]:
# X contains reverse_fingerprint + Ohe_df + water per ion and concentration of salt
X.head()

,mfp_0,mfp_1,mfp_2,mfp_3,mfp_4,mfp_5,mfp_6,mfp_7,mfp_8,mfp_9,...,ohe_3,ohe_4,ohe_5,ohe_6,ohe_7,ohe_8,ohe_9,ohe_10,Water-per-ion,concentration_of_salt_M
0,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.00
1,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.02
2,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.20
3,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.50
4,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,1.00


In [13]:
# y contains the rest of the parameter in the data
y.head()

,gr_minima_Ion_H2O,gr_peak_position_Ion_H2O,gr_peak_height_Ion_H2O,Nr_Ion_H2O,gr_minima_CG_H2O,gr_peak_position_CG_H2O,gr_peak_height_CG_H2O,Nr_CG_H2O_,gr_minima_CG_Ion,gr_peak_position_CG_Ion,gr_peak_height_CG_Ion,Nr_CG_Ion
0,4.45,3.65,6.5,4.7,5.4,3.55,2.16,5.3,6.25,4.15,5.6,1.9
1,4.45,3.65,6.5,4.7,5.5,3.55,2.30,6.1,6.15,4.15,5.8,1.9
2,4.45,3.65,6.3,4.6,5.5,3.55,1.90,5.0,6.15,4.15,5.8,2.0
3,4.45,3.65,6.2,4.6,5.5,3.55,1.80,4.5,6.15,4.15,5.6,2.2
4,4.55,3.65,6.0,4.7,5.0,3.55,1.50,3.0,6.15,4.15,5.1,2.5


In [23]:
Y = y['gr_minima_Ion_H2O']

### Split and normailize data

y is going to be y['gr_minima_Ion_H2O']

In [25]:
(data_solvation_scaler, X_train, X_test, y_train, y_test) = preprocess.normalizedata(X, y, splitRatio=0.2, state = 48, transform = True, property = 'Y')

In [27]:
y_train['gr_minima_Ion_H2O']

77   -0.470588
54   -1.000000
10   -0.517647
78   -0.470588
33   -0.529412
        ...   
48   -0.411765
68   -0.705882
81   -0.588235
51   -0.411765
0     1.000000
Name: gr_minima_Ion_H2O, Length: 73, dtype: float64

### PCA transformation for A1 and A2

Reminder the the input vestors:
- A0 represents all data with no transformation
- A1 PCA applied only to the polymer fingerprint
- A2 PCE applied to all input features

#### A1 (MDFP represent the polymer fingerprint input)

In [17]:
pca_X_train_MDFP, pca_X_test_MDFP = preprocess.convertInputsPCA(X_train, X_test, MFprints, n_comp=10, type = 'MDFP')

#### A2 (ALL represent all input)

In [18]:
pca_X_train_ALL, pca_X_test_ALL = preprocess.convertInputsPCA(X_train, X_test, MFprints, n_comp=10, type = 'All')

# ML models

## Support Vector Regression (SVR)

Testing the code for A2

In [29]:
solver = svrSolvers()
trainer = Trainers()

res_A2 = solver.svrSolver(X_train=pca_X_train_ALL,
                X_test=pca_X_test_ALL,
                y_train=y_train['gr_minima_Ion_H2O'],
                y_test=y_test['gr_minima_Ion_H2O'])

h_params_A2, best_F_A2 = solver.get_best(res_A2)

score = trainer.trainSVR_SingleOutput(h_params=h_params_A2,
                        X_train=X_train,
                        X_test=X_test,
                        y_train=y_train['gr_minima_Ion_H2O'],
                        y_test=y_test['gr_minima_Ion_H2O'],
                        folds=5)

{'C': 41.76049826978715, 'epsilon': 0.012286352598724284}
{'C': 72.0604168948716, 'epsilon': 6.053628126910477}
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        2 |      1 |             - |             -
{'C': 72.0604168948716, 'epsilon': 0.3009392332060581}
{'C': 41.76049826978715, 'epsilon': 6.069687443092774}
{'C': 52.864167896745705, 'epsilon': 5.949025304005646}
{'C': 72.0604168948716, 'epsilon': 0.5072755760661412}
     2 |        6 |      2 |  1.0000000000 |         ideal
{'C': 41.76049826978715, 'epsilon': 1.7926026381996865}
{'C': 66.10316060065897, 'epsilon': 0.3009392332060581}
{'C': 72.0604168948716, 'epsilon': 0.0298703801962584}
{'C': 78.65931170889557, 'epsilon': 0.013811636386431708}
     3 |       10 |      2 |  0.0135430177 |         ideal
{'C': 72.0604168948716, 'epsilon': 0.014409066255933356}
{'C': 72.45264929441929, 'epsilon': 0.3009392332060581}
{'C': 83.93058541673011, 'epsilon': 0.09201102307925474}
{'C': 75.53302025323181, 'epsilon': 

In [30]:
print(h_params_A2)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

{'C': 78.50740176970953, 'epsilon': 0.010110669079840923}
0.96 accuracy with a standard deviation of 0.03. R^2: 0.97


## Artificial Neural Network (ANN)

Testing the code for A2

In [31]:
solver = annSolvers()
trainer = Trainers()

res_A2 = solver.annSolver(X_train=pca_X_train_ALL,
                X_test=pca_X_test_ALL,
                y_train=y_train['gr_minima_Ion_H2O'],
                y_test=y_test['gr_minima_Ion_H2O'])

h_params_A2, best_F_A2 = solver.get_best(res_A2)

score = trainer.trainANN_SingleOutput(h_params=h_params_A2,
                        X_train=X_train,
                        X_test=X_test,
                        y_train=y_train['gr_minima_Ion_H2O'],
                        y_test=y_test['gr_minima_Ion_H2O'],
                        folds=5)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        2 |      2 |             - |             -


c:\Users\gabi1\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


     2 |        6 |      2 |  0.9704653934 |         ideal


c:\Users\gabi1\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


     3 |       10 |      2 |  0.0005911054 |             f
     4 |       14 |      2 |  0.0047019648 |         ideal


...

     8 |       30 |      2 |  1.0000000000 |         ideal


c:\Users\gabi1\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\gabi1\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\gabi1\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITE

     9 |       34 |      2 |  8.681751E+02 |         ideal


c:\Users\gabi1\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\gabi1\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\gabi1\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITE

    10 |       38 |      2 |  0.9845571237 |         ideal


In [33]:
print(h_params_A2)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

activation                 = ['logistic', 'tanh', 'relu']
solver                     = ['lbfgs', 'adam']
learning_rate              = ['constant', 'invscaling', 'adaptive']

{'alpha': 0.001, 'learning_rate_init': 0.01, 'activation_idx': 1, 'solver_idx': 0, 'learning_rate_idx': 1}
1.00 accuracy with a standard deviation of 0.00. R^2: 0.99


## Random Forest Regression (RFR)